# Notebook to run through all the steps in the pipeline

Very useful for debugging

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import rpy2.robjects as robjects
import multiprocessing
from experiments.pipelines.pipeline_rpy2 import Rpy2Pipeline

## Load analysis and project

In [3]:
project_id = 10
analysis_id = 38

In [4]:
project = Project.objects.get_or_create(id=project_id)[0]

DEBUG 2017-03-04 07:30:10,186 utils.py:91 | (0.009) SET SQL_AUTO_IS_NULL = 0; args=None
DEBUG 2017-03-04 07:30:10,190 utils.py:91 | (0.003) SELECT `projects_project`.`id`, `projects_project`.`title`, `projects_project`.`user_owner_id`, `projects_project`.`description`, `projects_project`.`created`, `projects_project`.`modified` FROM `projects_project` WHERE `projects_project`.`id` = 10 LIMIT 21; args=(10,)


In [5]:
analysis = Analysis.objects.get_or_create(id=analysis_id)[0]

DEBUG 2017-03-04 07:30:12,140 utils.py:91 | (0.004) SELECT `experiments_analysis`.`id`, `experiments_analysis`.`created`, `experiments_analysis`.`submited`, `experiments_analysis`.`owner`, `experiments_analysis`.`experiment_id`, `experiments_analysis`.`params_id`, `experiments_analysis`.`status` FROM `experiments_analysis` WHERE `experiments_analysis`.`id` = 38 LIMIT 21; args=(38,)


In [6]:
print analysis.status
analysis.status = 'Ready'
analysis.save()

DEBUG 2017-03-04 07:30:14,231 utils.py:91 | (0.001) UPDATE `experiments_analysis` SET `created` = '2017-02-28 20:27:40', `submited` = '2017-02-28 23:27:35', `owner` = 'admin', `experiment_id` = 17, `params_id` = 27, `status` = 'Ready' WHERE `experiments_analysis`.`id` = 38; args=(u'2017-02-28 20:27:40', u'2017-02-28 23:27:35', u'admin', 17, 27, 'Ready', 38)


Processing


## Create pipeline object

In [7]:
pipeline = Rpy2Pipeline(analysis, project)

INFO 2017-03-04 07:30:16,608 pipeline_rpy2.py:50 | ******************************************
INFO 2017-03-04 07:30:16,609 pipeline_rpy2.py:51 | Setup rpy2 connection
INFO 2017-03-04 07:30:16,611 pipeline_rpy2.py:52 | ******************************************


[1] "Initialising PuMP logging"


/home/pimp/pimp/venv/local/lib/python2.7/site-packages/rpy2/robjects/packages.py:63: UserWarning: Note: the specification for S3 class "AsIs" in package 'RJSONIO' seems equivalent to one from package 'BiocGenerics': not turning on duplicate class definitions for this class.

  return _reval(expr)

WARNING 2017-03-04 07:30:26,103 packages.py:63 | /home/pimp/pimp/venv/local/lib/python2.7/site-packages/rpy2/robjects/packages.py:63: UserWarning: Note: the specification for S3 class "AsIs" in package 'RJSONIO' seems equivalent to one from package 'BiocGenerics': not turning on duplicate class definitions for this class.

  return _reval(expr)

DEBUG 2017-03-04 07:30:27,907 utils.py:91 | (0.002) SELECT `experiments_experiment`.`id`, `experiments_experiment`.`title`, `experiments_experiment`.`created`, `experiments_experiment`.`modified` FROM `experiments_experiment` WHERE `experiments_experiment`.`id` = 17 LIMIT 21; args=(17,)
DEBUG 2017-03-04 07:30:27,926 utils.py:91 | (0.012) SELECT `fileu

[1] "PuMP log level set to DEBUG"
2017-03-04 07:30:25 ERROR:pump.misc.initialisePuMPLogging:Logging at ERROR level
2017-03-04 07:30:25 WARNING:pump.misc.initialisePuMPLogging:Logging at WARNING level
2017-03-04 07:30:25 INFO:pump.misc.initialisePuMPLogging:Logging at INFO level
2017-03-04 07:30:25 DEBUG:pump.misc.initialisePuMPLogging:Logging at DEBUG level
2017-03-04 07:30:27 INFO:PeakML.Isotope.plotSamples:OS type: x86_64-pc-linux-gnu
2017-03-04 07:30:27 INFO:PeakML.Isotope.plotSamples:Java parameters are: -Xmx8192m
2017-03-04 07:30:27 INFO:PeakML.Isotope.plotSamples:Using jar file /home/pimp/pimp/packrat/lib/x86_64-pc-linux-gnu/3.2.3/mzmatch.R/java/mzmatch-2.1-with-dependencies.jar
2017-03-04 07:30:27 INFO:PeakML.Isotope.plotSamples:Using SWT file /home/pimp/pimp/packrat/lib/x86_64-pc-linux-gnu/3.2.3/mzmatch.R/java/swt-4.5-gtk-linux-x86_64.jar
[1] "Initialising PiMP logging"
[1] "PiMP log level set to DEBUG"
2017-03-04 07:30:27 ERROR:Pimp.extra.initialisePiMPLogging:Logging at ERROR

DEBUG 2017-03-04 07:30:27,994 utils.py:91 | (0.001) SELECT `experiments_comparison`.`id`, `experiments_comparison`.`name`, `experiments_attributecomparison`.`attribute_id`, `groups_group`.`name`, `groups_attribute`.`name`, `experiments_attributecomparison`.`group` FROM `experiments_comparison` LEFT OUTER JOIN `experiments_attributecomparison` ON ( `experiments_comparison`.`id` = `experiments_attributecomparison`.`comparison_id` ) LEFT OUTER JOIN `groups_attribute` ON ( `experiments_attributecomparison`.`attribute_id` = `groups_attribute`.`id` ) LEFT OUTER JOIN `groups_group` ON ( `groups_attribute`.`group_id` = `groups_group`.`id` ) WHERE `experiments_comparison`.`experiment_id` = 17; args=(17,)
DEBUG 2017-03-04 07:30:28,028 utils.py:91 | (0.015) SELECT `experiments_database`.`name` FROM `experiments_database` INNER JOIN `experiments_params_databases` ON ( `experiments_database`.`id` = `experiments_params_databases`.`database_id` ) INNER JOIN `experiments_params` ON ( `experiments_para

2017-03-04 07:30:28 INFO:Pimp.extra.setPiMPLoggerAnalysisID:Setting analysis ID in logger to 38
2017-03-04 07:30:28 38:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Analysis id set


/home/pimp/pimp/venv/local/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Setting analysis directory to: analysis_38

  res = super(Function, self).__call__(*new_args, **new_kwargs)

WARNING 2017-03-04 07:30:28,698 functions.py:106 | /home/pimp/pimp/venv/local/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Setting analysis directory to: analysis_38

  res = super(Function, self).__call__(*new_args, **new_kwargs)



2017-03-04 07:30:28 38:INFO:Pimp.getAnalysisParams:Setting params
2017-03-04 07:30:28 38:DEBUG:Pimp.getAnalysisParams:Name: iqr Value: 0.5
2017-03-04 07:30:28 38:DEBUG:Pimp.getAnalysisParams:Name: rsd Value: 0.5
2017-03-04 07:30:28 38:DEBUG:Pimp.getAnalysisParams:Name: noise Value: 0.8
2017-03-04 07:30:28 38:DEBUG:Pimp.getAnalysisParams:Name: ppm Value: 3
2017-03-04 07:30:28 38:DEBUG:Pimp.getAnalysisParams:Name: mindetections Value: 3
2017-03-04 07:30:28 38:DEBUG:Pimp.getAnalysisParams:Name: minintensity Value: 5000
2017-03-04 07:30:28 38:DEBUG:Pimp.getAnalysisParams:Name: rtwindow Value: 0.05
2017-03-04 07:30:28 38:DEBUG:Pimp.getAnalysisParams:Name: rt.alignment Value: NA
2017-03-04 07:30:28 38:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Setting analysis ID in logger to 38
2017-03-04 07:30:28 38:INFO:Pimp.extra.setPiMPLoggerAnalysisID:Analysis id set
2017-03-04 07:30:28 38:INFO:Pimp.createAnalysisDir:Setting working dir to /home/pimp/media/projects/10:
2017-03-04 07:30:28 38:INFO:Pimp.cre

In [8]:
# shortcut ..
self = pipeline

In [9]:
xcms_params = self.get_value(self.pimp_params, 'xcms.params')
mzmatch_params = self.get_value(self.pimp_params, 'mzmatch.params')
peakml_params = self.get_value(self.pimp_params, 'peakml.params')
mzmatch_outputs = self.get_value(self.pimp_params, 'mzmatch.outputs')
mzmatch_filters = self.get_value(self.pimp_params, 'mzmatch.filters')
n_slaves = multiprocessing.cpu_count()

Here we only process the positive mode data

In [10]:
polarity = 'positive'

In [11]:
format_mzmatch_outputs = robjects.r['Pimp.getFormattedMzmatchOutputs']
formatted_mzmatch_outputs = format_mzmatch_outputs(self.analysis.id, polarity, mzmatch_outputs)
polarity_dir, combined_dir = self.create_input_directories(polarity, formatted_mzmatch_outputs)

### Peak detection and RT correction

In [12]:
print self.metadata.files[polarity]

[u'samples/POS/Beer_2_full1.mzXML', u'samples/POS/Beer_2_full2.mzXML', u'samples/POS/Beer_1_full1.mzXML', u'samples/POS/Beer_1_full2.mzXML']


In [13]:
self.create_peakml(polarity, polarity_dir, xcms_params, mzmatch_params,
                   peakml_params, mzmatch_outputs, n_slaves)

/home/pimp/pimp/venv/local/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: 'snow'


  res = super(Function, self).__call__(*new_args, **new_kwargs)

WARNING 2017-02-28 20:44:53,759 functions.py:106 | /home/pimp/pimp/venv/local/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: 'snow'


  res = super(Function, self).__call__(*new_args, **new_kwargs)

/home/pimp/pimp/venv/local/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from 'package:BiocGenerics':

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, clusterSplit, parApply, parCapply,
    parLapply, parRapply, parSapply


  res = super(Function, self).__call__(*new_args, **new_kwargs)

WARNING 2017-02-28 20:44:53,764 functions.py:106 | /home/pimp/pimp/venv/local/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following obj

Starting snow cluster with 8 local sockets.
Detecting features in file # 1 : Beer_2_full1.mzXML 
Detecting features in file # 2 : Beer_2_full2.mzXML 
Detecting features in file # 3 : Beer_1_full1.mzXML 
Detecting features in file # 4 : Beer_1_full2.mzXML 
center sample:  Beer_1_full2

DEBUG 2017-02-28 20:49:44,297 pipeline_rpy2.py:454 | Now creating /home/pimp/media/projects/10/analysis_38/positive/Beer_1_full1.peakml


 
Processing: Beer_2_full1  Beer_2_full2  Beer_1_full1  


DEBUG 2017-02-28 20:50:10,090 pipeline_rpy2.py:454 | Now creating /home/pimp/media/projects/10/analysis_38/positive/Beer_1_full2.peakml


2017-02-28 20:49:45 38:INFO:PeakML.xcms.write.SingleInstance:positive
2017-02-28 20:49:45 38:INFO:PeakML.xcms.write.SingleInstance:retrieving raw data
2017-02-28 20:49:45 38:INFO:PeakML.xcms.write.SingleInstance:- /home/pimp/media/projects/10/samples/POS/Beer_1_full1.mzXML
2017-02-28 20:50:10 38:INFO::11539 peaks exported.


DEBUG 2017-02-28 20:50:39,600 pipeline_rpy2.py:454 | Now creating /home/pimp/media/projects/10/analysis_38/positive/Beer_2_full1.peakml


2017-02-28 20:50:10 38:INFO:PeakML.xcms.write.SingleInstance:positive
2017-02-28 20:50:10 38:INFO:PeakML.xcms.write.SingleInstance:retrieving raw data
2017-02-28 20:50:10 38:INFO:PeakML.xcms.write.SingleInstance:- /home/pimp/media/projects/10/samples/POS/Beer_1_full2.mzXML
2017-02-28 20:50:39 38:INFO::11471 peaks exported.


DEBUG 2017-02-28 20:51:05,716 pipeline_rpy2.py:454 | Now creating /home/pimp/media/projects/10/analysis_38/positive/Beer_2_full2.peakml


2017-02-28 20:50:40 38:INFO:PeakML.xcms.write.SingleInstance:positive
2017-02-28 20:50:40 38:INFO:PeakML.xcms.write.SingleInstance:retrieving raw data
2017-02-28 20:50:40 38:INFO:PeakML.xcms.write.SingleInstance:- /home/pimp/media/projects/10/samples/POS/Beer_2_full1.mzXML
2017-02-28 20:51:05 38:INFO::11382 peaks exported.
2017-02-28 20:51:06 38:INFO:PeakML.xcms.write.SingleInstance:positive
2017-02-28 20:51:06 38:INFO:PeakML.xcms.write.SingleInstance:retrieving raw data
2017-02-28 20:51:06 38:INFO:PeakML.xcms.write.SingleInstance:- /home/pimp/media/projects/10/samples/POS/Beer_2_full2.mzXML
2017-02-28 20:51:31 38:INFO::12014 peaks exported.


### Matching

Generate groups for matching

In [12]:
non_empty = self.generate_combinations(polarity, combined_dir)

DEBUG 2017-03-04 07:30:38,676 utils.py:91 | (0.000) SET SQL_AUTO_IS_NULL = 0; args=None
DEBUG 2017-03-04 07:30:38,688 utils.py:91 | (0.011) SELECT `groups_group`.`name`, `groups_attribute`.`name`, `fileupload_sample`.`name` FROM `groups_group` LEFT OUTER JOIN `groups_attribute` ON ( `groups_group`.`id` = `groups_attribute`.`group_id` ) LEFT OUTER JOIN `groups_sampleattribute` ON ( `groups_attribute`.`id` = `groups_sampleattribute`.`attribute_id` ) LEFT OUTER JOIN `fileupload_sample` ON ( `groups_sampleattribute`.`sample_id` = `fileupload_sample`.`id` ) WHERE `groups_group`.`id` IN (SELECT U0.`id` FROM `groups_group` U0 INNER JOIN `groups_attribute` U1 ON ( U0.`id` = U1.`group_id` ) INNER JOIN `groups_sampleattribute` U2 ON ( U1.`id` = U2.`attribute_id` ) INNER JOIN `fileupload_sample` U3 ON ( U2.`sample_id` = U3.`id` ) WHERE U3.`project_id` = 10); args=(10,)
DEBUG 2017-03-04 07:30:38,696 pipeline_rpy2.py:477 | [beer_taste with 2 levels]
DEBUG 2017-03-04 07:30:38,698 pipeline_rpy2.py:48

In [15]:
for group_label, index, description, files, abspath in non_empty:
    print group_label, index, description, files, abspath

group_0 (0,) taste_good [u'Beer_1_full2', u'Beer_1_full1'] /home/pimp/media/projects/10/analysis_38/positive/combined/group_0
group_1 (1,) taste_bad [u'Beer_2_full1', u'Beer_2_full2'] /home/pimp/media/projects/10/analysis_38/positive/combined/group_1


Match peaks across samples in groups

In [16]:
out_files = self.generate_peaksets(polarity_dir, combined_dir, non_empty, mzmatch_params)

INFO 2017-02-28 20:51:37,670 pipeline_rpy2.py:281 | Processing /home/pimp/media/projects/10/analysis_38/positive/combined/group_0
INFO 2017-02-28 20:51:59,719 pipeline_rpy2.py:281 | Processing /home/pimp/media/projects/10/analysis_38/positive/combined/group_1


2017-02-28 20:51:37 38:INFO:PuMP.callToArgList.dispatchCall:mzmatch.ipeak.Combine -i /home/pimp/media/projects/10/analysis_38/positive/combined/group_0/Beer_1_full2.peakml,/home/pimp/media/projects/10/analysis_38/positive/combined/group_0/Beer_1_full1.peakml -o /home/pimp/media/projects/10/analysis_38/positive/combined/group_0.peakml -label group_0 -ppm 3 -rtwindow 30 -combination set
2017-02-28 20:51:59 38:INFO:PuMP.callToArgList.dispatchCall:mzmatch.ipeak.Combine -i /home/pimp/media/projects/10/analysis_38/positive/combined/group_1/Beer_2_full1.peakml,/home/pimp/media/projects/10/analysis_38/positive/combined/group_1/Beer_2_full2.peakml -o /home/pimp/media/projects/10/analysis_38/positive/combined/group_1.peakml -label group_1 -ppm 3 -rtwindow 30 -combination set


In [17]:
print out_files

['/home/pimp/media/projects/10/analysis_38/positive/combined/group_0.peakml', '/home/pimp/media/projects/10/analysis_38/positive/combined/group_1.peakml']


Filter each group

In [18]:
out_files = self.filter_peaksets(out_files, mzmatch_params)

2017-02-28 20:52:23 38:INFO:PuMP.callToArgList.dispatchCall:mzmatch.ipeak.filter.RSDFilter -i /home/pimp/media/projects/10/analysis_38/positive/combined/group_0.peakml -o /home/pimp/media/projects/10/analysis_38/positive/combined/group_0_rsd.peakml -rejected /home/pimp/media/projects/10/analysis_38/positive/combined/group_0_rsdrej.peakml -rsd 0.5
2017-02-28 20:52:42 38:INFO:PuMP.callToArgList.dispatchCall:mzmatch.ipeak.filter.RSDFilter -i /home/pimp/media/projects/10/analysis_38/positive/combined/group_1.peakml -o /home/pimp/media/projects/10/analysis_38/positive/combined/group_1_rsd.peakml -rejected /home/pimp/media/projects/10/analysis_38/positive/combined/group_1_rsdrej.peakml -rsd 0.5


In [19]:
print out_files

['/home/pimp/media/projects/10/analysis_38/positive/combined/group_0_rsd.peakml', '/home/pimp/media/projects/10/analysis_38/positive/combined/group_1_rsd.peakml']


Final combine

In [20]:
out_file = self.combine_final(out_files, mzmatch_params, formatted_mzmatch_outputs)

2017-02-28 20:53:06 38:INFO:PuMP.callToArgList.dispatchCall:mzmatch.ipeak.Combine -i /home/pimp/media/projects/10/analysis_38/positive/combined/group_0_rsd.peakml,/home/pimp/media/projects/10/analysis_38/positive/combined/group_1_rsd.peakml -o /home/pimp/media/projects/10/analysis_38/positive/final_combined.peakml -label  -ppm 3 -rtwindow 30 -combination set


In [21]:
print out_file

/home/pimp/media/projects/10/analysis_38/positive/final_combined.peakml


Final filter

In [22]:
out_file = self.filter_final(out_file, mzmatch_filters, mzmatch_params, formatted_mzmatch_outputs)

2017-02-28 20:53:27 38:INFO:PuMP.callToArgList.dispatchCall:mzmatch.ipeak.filter.NoiseFilter -i /home/pimp/media/projects/10/analysis_38/positive/final_combined.peakml -o /home/pimp/media/projects/10/analysis_38/positive/final_combined_nf.peakml -codadw 0.8
2017-02-28 20:53:39 38:INFO:PuMP.callToArgList.dispatchCall:mzmatch.ipeak.filter.SimpleFilter -i /home/pimp/media/projects/10/analysis_38/positive/final_combined_nf.peakml -o /home/pimp/media/projects/10/analysis_38/positive/final.combined_sf.peakml -ppm 3 -mindetections 3 -minintensity 5000


In [23]:
print out_file

/home/pimp/media/projects/10/analysis_38/positive/final.combined_sf.peakml


### Gap filling

In [24]:
out_file = self.gap_filling(out_file, peakml_params, formatted_mzmatch_outputs)

2017-02-28 20:53:52 38:INFO:PeakML.Methods.getRawDataPaths:Raw data file located at: /home/pimp/media/projects/10/samples/POS/Beer_1_full2.mzXML,/home/pimp/media/projects/10/samples/POS/Beer_1_full1.mzXML,/home/pimp/media/projects/10/samples/POS/Beer_2_full1.mzXML,/home/pimp/media/projects/10/samples/POS/Beer_2_full2.mzXML
2017-02-28 20:53:53 38:INFO:PeakML.Methods.getChromData:Loading peakML file in memory (it can take some time, sorry)
2017-02-28 20:53:57 38:INFO:PeakML.Methods.getChromData:Done in: 3.44399999999996 s
2017-02-28 20:54:08 38:INFO:PeakML.Methods.getPeakData:Extracting peak data from PeakMl file,
2017-02-28 20:54:13 38:INFO:PeakML.Methods.getPeakData:Peak data created in 5.149 s
2017-02-28 20:54:21 38:INFO:PeakML.GapFiller:Working on file: /home/pimp/media/projects/10/samples/POS/Beer_1_full2.mzXML
2017-02-28 20:54:30 38:INFO:PeakML.GapFiller:Working on file: /home/pimp/media/projects/10/samples/POS/Beer_1_full1.mzXML
2017-02-28 20:54:39 38:INFO:PeakML.GapFiller:Working

In [25]:
print out_file

/home/pimp/media/projects/10/analysis_38/positive/final_combined_gapfilled.peakml


### Related peaks

In [26]:
out_file, basepeak_file = self.related_peaks(out_file, mzmatch_params, formatted_mzmatch_outputs)

2017-02-28 20:55:10 38:INFO:PuMP.callToArgList.dispatchCall:mzmatch.ipeak.sort.RelatedPeaks -i /home/pimp/media/projects/10/analysis_38/positive/final_combined_gapfilled.peakml -o /home/pimp/media/projects/10/analysis_38/positive/final_combined_related.peakml -basepeaks /home/pimp/media/projects/10/analysis_38/positive/final_combined_basepeaks.peakml -ppm 3 -rtwindow 30


In [ ]:
print out_file
print basepeak_file

/home/pimp/media/projects/10/analysis_38/positive/final_combined_related.peakml
/home/pimp/media/projects/10/analysis_38/positive/final_combined_basepeaks.peakml


In [13]:
out_file = '/home/pimp/media/projects/10/analysis_38/positive/final_combined_related.peakml'
basepeak_file = '/home/pimp/media/projects/10/analysis_38/positive/final_combined_basepeaks.peakml'

In [15]:
databases = self.r_dbs

### Identification

In [16]:
raw_data = self.identify(polarity, out_file, databases, non_empty, mzmatch_params, formatted_mzmatch_outputs)

INFO 2017-03-04 07:34:22,060 pipeline_rpy2.py:378 | /home/pimp/media/projects/10/analysis_38/stds_db.xml is not found, setting stds.xml.db to NULL


2017-03-04 07:34:22 38:INFO:PuMP.callToArgList.dispatchCall:mzmatch.ipeak.util.Identify -i analysis_38/positive/final_combined_related.peakml -o analysis_38/positive/final_combined_related_identified_hmdb.peakml -ppm 3 -databases /home/pimp/pimp/packrat/lib/x86_64-pc-linux-gnu/3.2.3/PiMP/dbs/hmdb.xml -rtwindowrelative FALSE -adducts M+H,M+ACN+Na,M+Na,M+K,M+ACN+H
2017-03-04 07:35:00 38:INFO:PuMP.callToArgList.dispatchCall:mzmatch.ipeak.convert.ConvertToText -i analysis_38/positive/final_combined_related_identified_hmdb.peakml -o analysis_38/positive/mzmatch_output_hmdb.txt -annotations identification,ppm,adduct,relation.id,relation.ship,codadw,charge
2017-03-04 07:35:04 38:INFO:PuMP.callToArgList.dispatchCall:mzmatch.ipeak.util.Identify -i analysis_38/positive/final_combined_related.peakml -o analysis_38/positive/final_combined_related_identified_kegg.peakml -ppm 3 -databases /home/pimp/pimp/packrat/lib/x86_64-pc-linux-gnu/3.2.3/PiMP/dbs/kegg.xml -rtwindowrelative FALSE -adducts M+H,M+A